## 1. Detection of Propaganda Techniques in News Articles

Project Type: 2

Group: 19

Ahmed Haj Abdel Khaleq 8223727

Malissa Ekanayake 6927445

## 2. Timesheet

| Ahmed H.               | Task            | Hours spent     | Challenge |
|----------------------------|-----------------|-----------------|-----------------|
                |            | Learning PyTorch | 6  | Tried (and failed at) learning PyTorch in order to implement Logistic Regression using Pytorch |                           
|                  | Learning Keras         | 2          | Tried (and sort of succeeded at) learning Keras in order to implement Logistic Regression using Keras       |
|                  | Dataset Cleaning        | 3          | Made sense of and attempted to modify and clean up the dataset for Task 1 of our project using techniques such as NLP  |
|                  | Reading into undersampling and oversampling        | 1          | Task 2's dataset is smaller than that of Task 1's, so I looked into methods to improve that  |



| Malissa E.              | Task            | Hours spent        | Challenge |
|----------------------------|-----------------|-----------------|-----------------|
 |            | Read Article Ref[1] which talks about Decision Trees vs Logistic Regression | 2  | We haven't used Decision Trees in detail in class, so I had to learn what they are in terms of Machine Learning |                           
|                  | Implemented Decision Trees using SKLearn         | 5          | I looked at alternatives to use when implementing Decision Trees, but found that SKLearn would be the best for our project because we are already using a different implementation for Logistic Regression, and we have not implemented Decision Trees in any of the Notebooks       |
|                  | Dataset Cleaning        | 3          | Made sense of and attempted to modify and clean up the dataset for Task 2 of our project using techniques such as NLP  |
|                  | NLP experimentation       | 3          | Experimented with different NLP techniques on our Task 2 Dataset to make up for the fact that it's unbalanced |

## 3. Introduction

This project is based on the SemEval 2020 competetition, Task 11 "Detection of Propaganda Techniques in News Articles" [0]. It is divided into 2 tasks: 

Task 1: Detect whether or not a given sequence of characters is propaganda

Task 2: Given a sequence of characters that is propaganda, classify the type of propaganda that it belongs to.


The problem studied is, as the title says, detecting propaganda in news articles using machine learning. 

Context: Many news articles today, especially considering today's political climate, often employ propaganda techniques in their articles in order to sway the reader into thinking one way or another. Therefore, it is the reader's duty to make sure that they are not being tricked into holding a certain political opinion by being able to detect propaganda techniques in news article that they read. However, it is not an easy task to know when a news article contains propaganda in it. As such, if we could automatically detect when a news article contains propaganda, we would make it easier for readers to know which news sources to trust, and which to avoid.

Link to AI: In order to automate this task, we decided to use AI to automatically detect whether or not a news article contains propaganda. In this notebook, we are comparing Decision Trees and 2 different implementations of Logistic Regression.

Why is it important: It's important because it helps users get better sources for news, and to know whether or not a news article is worth reading. It also helps news article authors to up the quality of their articles, because even people who write news articles have biases.



## 4. Dataset Description

Where does it come from: https://propaganda.qcri.org/semeval2020-task11/ (note that you need to register first, and then they will email you the dataset)

What does it contain: A baselines file, A datasets file, and a tools file. 

The baselines file contains baselines which ensure that the dataset has been labelled correctly, and the tools folder contains tools which help with organizing the results in a way that is fit for the actual competition. (Not important for the project)

The datasets file contains:

1) a README file which explains how the data is laid out. 

2) The train-articles folder in which the articles themselves are stored in as .txt files 

3) A train-labels-task1 folder which contains labels for each article where the article name, character on which propaganda starts, and character on which propaganda ends are specified

4) A train-lables-task2 folder which contains labels for each article where the article name, type of propaganda, character on which propaganda starts, and character on which propaganda ends are specified

A test set was not included (will be released in March), so we had to adopt a technique that was used in the notebook where we split the dataset, 90% to 10%. 


## Size:

Overall: 550 news articles, but when treating the sentences themselves as input, the dataset becomes of size 10940 

For Task 1: Using the input dataset, we had 5470 propaganda sentences. However, by adding the same amount sentences which were NOT tagged as propaganda, we effictively doubled the dataset and made it balanced (So no need to oversample/undersample). The final size of the dataset for Task 1 is 10940.

For Task 2: Since Task 2 only works on sentences labelled as propaganda, the dataset size for Task 2 is 5470. (We have looked at undersampling and oversampling, however we decided to try NLP techniques on it instead because it's something that we haven't tried before)


## Was cleaning required?

Since the dataset was all over the place (in 3 different folders, and even then the labels weren't even in the same place as the news articles!), a lot of cleaning was required. 

**Step 1A: First off, we had to gather the labels from the Task 1 folder, read them, and place them into a dict type (propTagsSpan) where the article number is the key, and a list containing the characters where the propaganda starts and ends as values.**

In [11]:
import os
newsArticles = os.listdir("datasets/train-articles/") # this is where our news articles are located
propagandaTagsSpan = os.listdir("datasets/train-labels-task1-span-identification") # this is where our tags are located
newsArticles.sort()
propagandaTagsSpan.sort()
propTagsSpan = {} # Dictionary containing the news article number as a key, and propoganda snip as values

for i in range(len(newsArticles)):
    article = newsArticles[i]
    articleNoExt = os.path.splitext(article)[0] # remove the .txt file extension ([2])
    newsArticles[i] = articleNoExt # replace newsArticles[i] with the same name but without the .txt extension
    articleNo = articleNoExt.replace('article', '') # remove 'article' to leave just the number
    
    tagPath = "datasets/train-labels-task1-span-identification/"+ articleNoExt + ".task1-SI.labels"
    with open(tagPath) as f:
        tags = f.readlines()
        # replace \t and \n in tags with " " for easier processing later on
        for i in range(len(tags)):
            tag = tags[i]
            tag = tag.replace("\t", " ")
            tag = tag.replace("\n", " ")
            tags[i] = tag 
        propTagsSpan[articleNoExt] =  tags
    f.close()

print(propTagsSpan[newsArticles[0]])

['111111111 265 323 ', '111111111 1795 1935 ', '111111111 149 157 ', '111111111 1069 1091 ', '111111111 1334 1462 ', '111111111 1577 1616 ', '111111111 2023 2086 ']


**Step 1B: Do the same thing as 1A, but for Task 2, from the Task 2 dataset folder.**

In [12]:
propagandaTagsTechnique = os.listdir("datasets/train-labels-task2-technique-classification") # this is where our tags are located for techique
## note: in span-identification, techniques that overlap are not split
propagandaTagsTechnique.sort()
propTagsTechnique = {} # Dictionary containing news article as key, and technique as value

for i in range(len(newsArticles)):
    article = newsArticles[i]
    articleNoExt = os.path.splitext(article)[0] # remove the .txt file extension ([2])
    newsArticles[i] = articleNoExt # replace newsArticles[i] with the same name but without the .txt extension
    articleNo = articleNoExt.replace('article', '') # remove 'article' to leave just the number
    
    tagPath = "datasets/train-labels-task2-technique-classification/"+ articleNoExt + ".task2-TC.labels"
    
    with open(tagPath) as f:
        tags = f.readlines()
        # replace \t and \n in tags with " " for easier processing later on
        for i in range(len(tags)):
            tag = tags[i]
            tag = tag.replace(articleNo, " ")
            tag = tag.replace("\t", " ")
            tag = tag.replace("\n", " ")
            tags[i] = tag 
            #print(tag)
        propTagsTechnique[articleNoExt] = tags
    f.close()
    
print(propTagsTechnique[newsArticles[0]])


['  Appeal_to_Authority 265 323 ', '  Appeal_to_Authority 1795 1935 ', '  Doubt 149 157 ', '  Repetition 1069 1091 ', '  Appeal_to_fear-prejudice 1334 1462 ', '  Appeal_to_fear-prejudice 1577 1616 ', '  Appeal_to_fear-prejudice 1856 1910 ', '  Appeal_to_fear-prejudice 2023 2086 ']


**Step 2 : Using the dict created in Step 1, read all of the sentences that have been annotated as "propaganda" from the 'train-articles' folder, and put them in a list which will be named 'propSentencesSpan'.**

In [13]:
propSentencesSpan = []

for article in newsArticles:
    artPath = "datasets/train-articles/" + article + ".txt"
    
    tags = propTagsSpan[article]
    
    with open(artPath, encoding="utf-8") as f:
        wholeArticle = f.read()
        for tag in tags:
            tag = tag.split()
            start = int(tag[1])
            end = int(tag[2])
            
            taggedLine = wholeArticle[start:end]
            taggedLine = taggedLine.replace("\n", " ")
            taggedLine = taggedLine.replace("\t", " ")
          
            propSentencesSpan.append(taggedLine)
    f.close()
    
print(propSentencesSpan[0])


The next transmission could be more pronounced or stronger


**Step 3: Create a dictionary with the keys being the propoganda sentences, and the values being their associated propoganda type. This is to setup our data to be put into a Pandas dataframe for Task 2.**

In [14]:
# dictionary with sentences as keys and technique as value
propagandaTechniques = {}
propagandaTechniques["Sentence"] = []
propagandaTechniques["Technique"] = []

for article in newsArticles:
    artPath = "datasets/train-articles/" + article + ".txt"
    
    tags = propTagsTechnique[article]
    with open(artPath, encoding="utf-8") as f:
        wholeArticle = f.read()
        for tag in tags:
            tag = tag.split()
            propagandaTechniques["Technique"].append(tag[0]) # add technique to dictionary
            #print(tag[0])
            start = int(tag[1])
            end = int(tag[2])
            taggedLine = wholeArticle[start:end]
            taggedLine = taggedLine.replace("\n", " ")
            taggedLine = taggedLine.replace("\t", " ")
            #print(article + " " + taggedLine)
            propagandaTechniques["Sentence"].append(taggedLine) # add snippet to dictionary
    f.close()
    


**Step 4: Using the list of propaganda senteces that we've gathered, create another list, 'notPropSentences' which will contain sentences from the articles that have not been annotated as propaganda. This will be used for Task 1.**

Note: Since we're relying on the actual news article now instead of labelled characters, we need to sentence_tokenize each article, and go through each sentence. If a sentence or phrase that is in our list of propaganda sentences is inside or the same as the current sentence, then it does not get added to the list of non propaganda sentences and vice versa.

In [15]:
import nltk
notPropSentences = []

count = 0
maxNum = len(propSentencesSpan) # we want an equal number of propaganda and non-propaganda sentences to create a balanced training set
for article in newsArticles:
    artPath = "datasets/train-articles/" + article + ".txt"
    with open(artPath, encoding="utf-8") as f:
        wholeArticle = f.read()
        
        # Remove SPANNED lines of propoganda from articles to detect non-propoganda lines
        currentPropSentences = []
        tags = propTagsSpan[article]
        for tag in tags:
            tag = tag.split()
            start = int(tag[1])
            end = int(tag[2])
            taggedLine = wholeArticle[start:end]
            taggedLine = taggedLine.replace("\n", " ")
            taggedLine = taggedLine.replace("\t", " ")
            currentPropSentences.append(taggedLine)
        
        sentences = nltk.sent_tokenize(wholeArticle)
        for sentence in sentences:
            if(count == maxNum):
                break
            notProp = True
            sentence = sentence.replace("\n", " ")
            sentence = sentence.replace("\t", " ")
            for propSentence in currentPropSentences:
                if(propSentence in sentence):
                    notProp = False
                    
            if(notProp): 
                count +=1
                notPropSentences.append(sentence)

print(len(propSentencesSpan))
print(len(notPropSentences))
print(notPropSentences[0])

5470
5470
An outbreak of both bubonic plague, which is spread by infected rats via flea bites, and pneumonic plague, spread person to person, has killed more than 200 people in the Indian Ocean island nation since August.


**Step 5A: Merge both 'propSentences' and 'notPropSentences' into one list which will serve as our dataset for Task 1.**

Note: At first, we attempted to do POS tagging on the sentences, and then use a Vectorizer on the POS tagged sentences, and use that instead of just doing a Vectorizer on the sentences directly. However we found that using a countVectorizer on the sentences as they are yielded higher accuracy. We believe this is because when it comes to detecting propaganda, the words used are more important than the structure of the sentence itself because some words are more prevalent in propaganda sentences than in non-propaganda sentences

In [16]:
import pandas as pd

# In order to use pandas, we have to create a dict where we will store as values, which we can then convert into a Pandas DataFrame
sentencesToCSV = {}
sentencesToCSV["Propaganda"] = []
sentencesToCSV["Sentence"] = []

# A special dict for the Keras Logistic Regression Model:

sentencesToCSVKeras = {}
sentencesToCSVKeras["Propaganda"] = []
sentencesToCSVKeras["Sentence"] = []

for sentence in propSentencesSpan: 
    sentencesToCSV["Propaganda"].append("Yes")
    sentencesToCSV["Sentence"].append(sentence) 
    
    sentencesToCSVKeras["Propaganda"].append(1)
    sentencesToCSVKeras["Sentence"].append(sentence)
    

for sentence in notPropSentences:  
    sentence.replace("\n", " ")
    sentence.replace("\t", " ")
    
    sentencesToCSV["Propaganda"].append("No")
    sentencesToCSV["Sentence"].append(sentence) 
    
    sentencesToCSVKeras["Propaganda"].append(0)
    sentencesToCSVKeras["Sentence"].append(sentence)


df = pd.DataFrame.from_dict(sentencesToCSV)
dfKeras = pd.DataFrame.from_dict(sentencesToCSVKeras)

df.head()



,Propaganda,Sentence
0,Yes,The next transmission could be more pronounced...
1,Yes,when (the plague) comes again it starts from m...
2,Yes,appeared
3,Yes,"a very, very different"
4,Yes,He also pointed to the presence of the pneumon...


**Step 5B: Create the Pandas dataframe for Task 2 from the dict that we made in Step 3.**

In [17]:
# Pandas DataFrame for propoganda snippet and propoganda technique

df_techniques = pd.DataFrame.from_dict(propagandaTechniques)
df_techniques.head()
    

,Sentence,Technique
0,The next transmission could be more pronounced...,Appeal_to_Authority
1,when (the plague) comes again it starts from m...,Appeal_to_Authority
2,appeared,Doubt
3,"a very, very different",Repetition
4,He also pointed to the presence of the pneumon...,Appeal_to_fear-prejudice


**Step 6A: Similar to what we have done in Notebook 3, we are going to split the data such that 90% is used for training and 10% is used for testing for Task 1.**

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_tags, test_tags = train_test_split(df["Sentence"],
                                                                      df["Propaganda"],
                                                                      test_size=0.1, 
                                                                      random_state=10,
                                                                      stratify=df["Propaganda"])

train_tags = train_tags.to_numpy()
train_sentences = train_sentences.to_numpy()
# Testing set (what we will use to test the trained model)
test_tags = test_tags.to_numpy()
test_sentences = test_sentences.to_numpy()


print(train_sentences[1])
print(train_tags[1])


# Do the same thing for the Keras df

train_sentences, test_sentences, train_tags, test_tags = train_test_split(dfKeras["Sentence"],
                                                                      dfKeras["Propaganda"],
                                                                      test_size=0.1, 
                                                                      random_state=10,
                                                                      stratify=dfKeras["Propaganda"])

train_tags_keras = train_tags.to_numpy()
train_sentences_keras = train_sentences.to_numpy()
# Testing set (what we will use to test the trained model)
test_tags_keras = test_tags.to_numpy()
test_sentences_keras = test_sentences.to_numpy()


I think that will reveal a whole other layer of who knew what, where and when.
No


**Step 6B: Splitting the data such that 90% is used for training and 10% is used for testing for Task 2.**

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

train2_sentences, test2_sentences, train2_tags, test2_tags = train_test_split(df_techniques["Sentence"],
                                                                      df_techniques["Technique"],
                                                                      test_size=0.1, 
                                                                      random_state=10,
                                                                      stratify=df_techniques["Technique"])

train2_tags = train2_tags.to_numpy()
train2_sentences = train2_sentences.to_numpy()
# Testing set (what we will use to test the trained model)
test2_tags = test2_tags.to_numpy()
test2_sentences = test2_sentences.to_numpy()


print(train2_sentences[40])
print(train2_tags[40])

White House Leaker
Name_Calling,Labeling


**Step 7A: Vectorizer on the df for Task 1.**

In [20]:
from sklearn.feature_extraction.text import CountVectorizer


count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train_sentences)
test_counts = count_vect.transform(test_sentences)
print(train_counts.shape)
print(train_tags.shape)


# Same thing but for Keras

count_vect_keras = CountVectorizer()
train_counts_keras = count_vect_keras.fit_transform(train_sentences_keras).toarray()
test_counts_keras = count_vect_keras.transform(test_sentences_keras).toarray()

(9846, 13239)
(9846,)


**Step 7B: Vectorizer on the df for Task 2**

In [21]:
count_vect2 = CountVectorizer()
train2_counts = count_vect2.fit_transform(train2_sentences)
test2_counts = count_vect2.transform(test2_sentences)

# For Keras. Transforms String labels to integer. Basically a one hot encoder.
from sklearn.preprocessing import LabelBinarizer # from [4]

encoder = LabelBinarizer()

train2_tags_keras = encoder.fit_transform(train2_tags)
test2_tags_keras = encoder.fit_transform(test2_tags)


## 5. Details of Methods (Approaches) Used

## 5A. Logistic Regression

A) Logistic Regression: Logistic Regression fits a single line to divide the space exactly into two (binomial LR). [1]

For Task 1, Binomial Logistic Regression was used. For Task 2, Multinomial Logistic Regression was used because we need to divide the space into more than 2 parts since we have multiple classes instead of only 2 classes.

B) The reason why we decided to use Logistic Regression is because we think it would function really well for the goal of the project. Since our project is divided into 2 Tasks, a Binary Classification Task and a Multinomial Classification. We believe that it will perform better than Decision Trees in Task 1, while for Task 2, we think their performance would be close to each other's.

C) Keras was used just to learn a new package. But we ran into some issues with its implementation so we are also using SKlearn just so that our knowledge (or lack of) when implementing Logistic Regression using Keras isn't a factor. 


(Initially we were using PyTorch, but it we couldn't get it to work at all so we used an SKlearn alternative just so that we can finish the project and not waste time) 


    

Setting up the Keras Binomial Logistic Model

In [22]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras import backend as K

# The functions below were taken from [3]
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    



keras_lr_1 = Sequential() 
keras_lr_1.add(Dense(input_dim = 13239, units = 1)) # 13229 is the shape of the df for task 1, 1 is output dimension of the test tag which is 0 or 1 
keras_lr_1.add(Activation('relu'))
keras_lr_1.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy', recall_m, precision_m])

Using TensorFlow backend.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Setting up the SKLearn Binomial Logistic Regression Model 

In [23]:
from sklearn.linear_model import LogisticRegression
import datetime
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

# What we will use for LogisticRegression
clf_lr = LogisticRegression(solver='lbfgs', multi_class="ovr", max_iter=1000, random_state=1)

Setting up the Multinomial Keras Logistic Regression Model

In [24]:
keras_lr_2 = Sequential() 
keras_lr_2.add(Dense(input_dim = 7275, units = 14)) # 13229 is the shape of the df, 1 is output dimension of the test tag which is 0 or 1 
keras_lr_2.add(Activation('relu'))
keras_lr_2.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy', recall_m, precision_m])

Setting up the Multinomial SKLearn Logistic Regression Model

In [25]:
# What we will use for LogisticRegression
clf_lr2 = LogisticRegression(solver='lbfgs', multi_class="multinomial", max_iter=1000, random_state=1)

Defining the training method for the SKLearn Logistic Regression Models (from the notebooks):

In [26]:
def train_model(clf, X_train, y_train, epochs=10):
    scores = []
    print("Starting training...")
    for i in range(1, epochs + 1):
        print("Epoch:" + str(i) + "/" + str(epochs) + " -- " + str(datetime.datetime.now()))
        clf.fit(X_train, y_train)
        score = clf.score(X_train, y_train)
        scores.append(score)
    print("Done training.")
    return scores

Defining methods to calculate precision and recall for SKLearn (from notebooks)

In [27]:
def precision(actualTags, predictions, classOfInterest):
    actualCounter = 0
    predCounter = 0
    for i in range(len(predictions)):
        if classOfInterest == predictions[i]:
            predCounter += 1
            if classOfInterest == actualTags[i]:
                actualCounter += 1
    return actualCounter/predCounter

def recall(actualTags, predictions, classOfInterest):
    actualTagCounter = 0
    predictionsCounter = 0
    for i in range(len(predictions)):
        if classOfInterest == actualTags[i]:
            actualTagCounter += 1
            if classOfInterest == predictions[i]:
                predictionsCounter += 1
   
    return predictionsCounter/actualTagCounter



Training the Keras Binomial LR model and printing the accuracy, precision, and recall:

In [28]:
keras_lr_1.fit(train_counts, train_tags_keras, epochs= 1, batch_size=128, verbose=1, validation_data=(test_counts, test_tags_keras))

loss, accuracy1_keras, recall1_keras, precision1_keras = keras_lr_1.evaluate(test_counts, test_tags_keras, verbose=0)

print("Accuracy:", accuracy1_keras)
print("Precision:", precision1_keras)
print("Recall:", recall1_keras)
      




Train on 9846 samples, validate on 1094 samples
Epoch 1/1





9846/9846 [==============================] - 2s 176us/step - loss: 0.7594 - acc: 0.5952 - recall_m: 0.3465 - precision_m: 0.5644 - val_loss: 0.6281 - val_acc: 0.7559 - val_recall_m: 0.7225 - val_precision_m: 0.7736
Accuracy: 0.7559414990859232
Precision: 0.7743745277307151
Recall: 0.7208834043150628


Training the SKLearn Binomial LR model and printing the accuracy, recall, and precision:

In [29]:
clf_lr_score = train_model(clf_lr, train_counts, train_tags, 1)
y_pred = clf_lr.predict(test_counts)
print("Accuracy:",clf_lr_score)



Starting training...
Epoch:1/1 -- 2019-12-06 23:33:35.803248
Done training.
Accuracy: [0.9313426772293317]


Training the Keras Multinomial LR Model and printing the score:

In [30]:
keras_lr_2.fit(train2_counts, train2_tags_keras, epochs= 1, batch_size=128, verbose=1, validation_data=(test2_counts, test2_tags_keras))

loss, accuracy2_keras, recall2_keras, precision2_keras = keras_lr_2.evaluate(test2_counts, test2_tags_keras, verbose=0)

print("Accuracy:", accuracy2_keras)
print("Precision:", precision2_keras)
print("Recall:", recall2_keras)


Train on 5732 samples, validate on 637 samples
Epoch 1/1
5732/5732 [==============================] - 1s 194us/step - loss: 2.6675 - acc: 0.2406 - recall_m: 0.0686 - precision_m: 0.1063 - val_loss: 2.1430 - val_acc: 0.3203 - val_recall_m: 0.1256 - val_precision_m: 0.1343
Accuracy: 0.3202511773472491
Precision: 0.13707748666008002
Recall: 0.12558869699387573


In [31]:
clf_lr_score2 = train_model(clf_lr2, train2_counts, train2_tags, 1)
print(clf_lr_score2)


Starting training...
Epoch:1/1 -- 2019-12-06 23:33:37.561765
Done training.
[0.8565945568736916]


## 5B. Decision Trees

A) Decision Trees: Decision Trees bisect the space into smaller and smaller regions. [1]


B) The reason why we decided to use Decision Trees is because it's a discriminative learner just like Logistic Regression. However it's designed with multinomial classification in mind, so it's interesting to see how it compares to Binomial Logistic Regression in Task 1. (We predict that Binomial LR will perform better in Task 1, while they will perform similarly in Task 2)

C) SKLearn was used to implement Decision Trees to keep things simple. We chose SKLearn because Decision Trees are already a new kind of discriminative learner to us, so we did not want to choose something too complicated when implementing them



**Implementing and printing the score, precision, and recall for precision trees for Task 1:**


In [32]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.metrics import average_precision_score

# Create Decision Tree classifer object
clf_dt = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf_dt = clf_dt.fit(train_counts,train_tags)

#Predict the response for test dataset
y_pred = clf_dt.predict(test_counts)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_tags, y_pred))

print("Precision:", precision(test_tags, y_pred, "Yes"))
print("Recall:", recall(test_tags, y_pred, "Yes"))


Accuracy: 0.7020109689213894


ZeroDivisionError: division by zero

**Implementing and printing the score, precision, and recall for precision trees for Task 2:**

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.metrics import average_precision_score

# Create Decision Tree classifer object
clf_dt2 = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf_dt2 = clf_dt2.fit(train2_counts,train2_tags)

#Predict the response for test dataset
y_pred = clf_dt2.predict(test2_counts)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(test2_tags, y_pred))

print("Precision:", precision(test2_tags, y_pred, "Name_Calling,Labeling"))
print("Recall:", recall(test2_tags, y_pred, "Name_Calling,Labeling"))


This is where our usage of the Keras models stopped. The scores for the Keras models were significantly lower than that of their SKLearn counterparts, and we believe it might be because our implementation was poor, so we decided to drop it entirely.

## 6. Feature Engineering

A) Our datasets consisted of the sentence and whether or not it's propaganda for Task 1. and For Task 2, the dataset consisted of the sentence, and the type of propaganda used in the sentence. 

A new version of both the train and the test sentences are created with any non-alphanumeric tokens and any *stopwords* removed for Task 2. 

B) Applying NLP Techique 'Removal of Stopwords and Non-Alphanumerical Characters for Task 2 Dataset. A stopword is a word that is too common to be deemed useful when training or testing a model. In nltk, we will use the stopwords list available as a guide on what to remove. We believe that the structure of a sentence is more important than the actual words used to make up a sentence.

C) No Features got removed because our dataset consists only of sentences and its propaganda status for task 1, or propaganda type for task 2. 

# Applying NLP Techniques on our already made dataset

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords

Removing Stopwords and non-alphanumeric characters (from the notebooks)

In [ ]:
import re

# From a given train or test set, returns a new version of the set (as a numpy array)
# with any stopwords or non-alphanumeric characters removed
def no_alpnum_no_stopword(reviews):
    new_reviews = []
    for review in reviews:
        # Tokenize
        tokens = word_tokenize(review.lower())
        # Remove non-alphanumeric characters
        review_tok_alpha = [t for t in tokens if re.match("^[a-zA-Z]+$", t)]
        # Remove stopwords after removing non-alphanumeric characters
        #review_stopwords = []
        #for t in review_tok_alpha:
        #    if t not in stopwords.words('english'):
        #        review_stopwords.append(t)
        review_stopwords = [t for t in review_tok_alpha if t not in stopwords.words('english')]
        # Re-form the tokens
        review_text = " ".join(review_stopwords)
        #print(review_text)
        # Append to new_reviews
        new_reviews.append(review_text)
    return (new_reviews)

Applying the function on our dataset:

In [ ]:
# New version of the train set (just modifying the sentences, not the tags)
train2_sentences_sw = no_alpnum_no_stopword(train2_sentences)
#print(train_sentences_sw[0])
# New version of the test set (just modifying the sentences, not the tags)
test2_sentences_sw = no_alpnum_no_stopword(test2_sentences)
#print(test_sentences_sw[0])

## 7. Analysis of Results

A) Evaluation methods used are accuracy, precision, and recall for each propaganda type

## B) Individual Results

### Logistic Regression

Repeat earlier steps from 5. when setting up and training multinomial Logistic Regression Model:

In [ ]:
count_vect_sw = CountVectorizer()
train2_counts_sw = count_vect_sw.fit_transform(train2_sentences_sw)
test2_counts_sw = count_vect_sw.transform(test2_sentences_sw)

clf_lr2_sw = LogisticRegression(solver='lbfgs', multi_class="multinomial", max_iter=1000, random_state=1)

clf_lr2_scores_sw = train_model(clf_lr2_sw, train2_counts_sw, train2_tags, 1)
print(clf_lr2_scores_sw)

Testing on test set.

In [ ]:
print("Accuracy: ", clf_lr2_sw.score(test2_counts_sw, test2_tags))

def precision(actualTags, predictions, classOfInterest):
    actualCounter = 0
    predCounter = 0
    for i in range(len(predictions)):
        if classOfInterest == predictions[i]:
            predCounter += 1
            if classOfInterest == actualTags[i]:
                actualCounter += 1
    if (predCounter == 0 ):
        return 0
    else:
        return actualCounter/predCounter

def recall(actualTags, predictions, classOfInterest):
    actualTagCounter = 0
    predictionsCounter = 0
    for i in range(len(predictions)):
        if classOfInterest == actualTags[i]:
            actualTagCounter += 1
            if classOfInterest == predictions[i]:
                predictionsCounter += 1
                
    if (actualTagCounter == 0):
        return 0
    else:
        return predictionsCounter/actualTagCounter


y_pred = clf_lr2_sw.predict(test2_counts_sw)

print("\nPrecision Values for Each Class:")
print("Appeal_to_Authority: ", precision(test2_tags, y_pred, "Appeal_to_Authority"))
print("Doubt: ", precision(test2_tags, y_pred, "Doubt"))
print("Repetition: ", precision(test2_tags, y_pred, "Repetition"))
print("Appeal_to_fear-prejudice: ", precision(test2_tags, y_pred, "Appeal_to_fear-prejudice"))
print("Slogans: ", precision(test2_tags, y_pred, "Slogans"))
print("Black-and-White_Fallacy: ", precision(test2_tags, y_pred, "Black-and-White_Fallacy"))
print("Loaded_Language: ", precision(test2_tags, y_pred, "Loaded_Language"))
print("Flag-Waving: ", precision(test2_tags, y_pred, "Flag-Waving"))
print("Name_Calling,Labeling: ", precision(test2_tags, y_pred, "Name_Calling,Labeling"))
print("Whataboutism,Straw_Men,Red_Herring: ", precision(test2_tags, y_pred, "Whataboutism,Straw_Men,Red_Herring"))
print("Causal_Oversimplification: ", precision(test2_tags, y_pred, "Causal_Oversimplification"))
print("Exaggeration,Minimisation: ", precision(test2_tags, y_pred, "Exaggeration,Minimisation"))
print("Bandwagon,Reductio_ad_hitlerum: ", precision(test2_tags, y_pred, "Bandwagon,Reductio_ad_hitlerum"))
print("Thought-terminating_Cliches: ", precision(test2_tags, y_pred, "Thought-terminating_Cliches"))

print("\nRecall Values for Each Class:")
print("Appeal_to_Authority: ", recall(test2_tags, y_pred, "Appeal_to_Authority"))
print("Doubt: ", recall(test2_tags, y_pred, "Doubt"))
print("Repetition: ", recall(test2_tags, y_pred, "Repetition"))
print("Appeal_to_fear-prejudice: ", recall(test2_tags, y_pred, "Appeal_to_fear-prejudice"))
print("Slogans: ", recall(test2_tags, y_pred, "Slogans"))
print("Black-and-White_Fallacy: ", recall(test2_tags, y_pred, "Black-and-White_Fallacy"))
print("Loaded_Language: ", recall(test2_tags, y_pred, "Loaded_Language"))
print("Flag-Waving: ", recall(test2_tags, y_pred, "Flag-Waving"))
print("Name_Calling,Labeling: ", recall(test2_tags, y_pred, "Name_Calling,Labeling"))
print("Whataboutism,Straw_Men,Red_Herring: ", recall(test2_tags, y_pred, "Whataboutism,Straw_Men,Red_Herring"))
print("Causal_Oversimplification: ", recall(test2_tags, y_pred, "Causal_Oversimplification"))
print("Exaggeration,Minimisation: ", recall(test2_tags, y_pred, "Exaggeration,Minimisation"))
print("Bandwagon,Reductio_ad_hitlerum: ", recall(test2_tags, y_pred, "Bandwagon,Reductio_ad_hitlerum"))
print("Thought-terminating_Cliches: ", recall(test2_tags, y_pred, "Thought-terminating_Cliches"))

### Decision Trees

Repeat earlier steps from 5. for Decision Trees

In [ ]:
# Create Decision Tree classifer object
clf_dt2_sw = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf_dt2_sw = clf_dt2_sw.fit(train2_counts_sw,train2_tags)

#Predict the response for test dataset
y_pred = clf_dt2_sw.predict(test2_counts_sw)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(test2_tags, y_pred))

print("\nPrecision Values for Each Class:")
print("Appeal_to_Authority: ", precision(test2_tags, y_pred, "Appeal_to_Authority"))
print("Doubt: ", precision(test2_tags, y_pred, "Doubt"))
print("Repetition: ", precision(test2_tags, y_pred, "Repetition"))
print("Appeal_to_fear-prejudice: ", precision(test2_tags, y_pred, "Appeal_to_fear-prejudice"))
print("Slogans: ", precision(test2_tags, y_pred, "Slogans"))
print("Black-and-White_Fallacy: ", precision(test2_tags, y_pred, "Black-and-White_Fallacy"))
print("Loaded_Language: ", precision(test2_tags, y_pred, "Loaded_Language"))
print("Flag-Waving: ", precision(test2_tags, y_pred, "Flag-Waving"))
print("Name_Calling,Labeling: ", precision(test2_tags, y_pred, "Name_Calling,Labeling"))
print("Whataboutism,Straw_Men,Red_Herring: ", precision(test2_tags, y_pred, "Whataboutism,Straw_Men,Red_Herring"))
print("Causal_Oversimplification: ", precision(test2_tags, y_pred, "Causal_Oversimplification"))
print("Exaggeration,Minimisation: ", precision(test2_tags, y_pred, "Exaggeration,Minimisation"))
print("Bandwagon,Reductio_ad_hitlerum: ", precision(test2_tags, y_pred, "Bandwagon,Reductio_ad_hitlerum"))
print("Thought-terminating_Cliches: ", precision(test2_tags, y_pred, "Thought-terminating_Cliches"))

print("\nRecall Values for Each Class:")
print("Appeal_to_Authority: ", recall(test2_tags, y_pred, "Appeal_to_Authority"))
print("Doubt: ", recall(test2_tags, y_pred, "Doubt"))
print("Repetition: ", recall(test2_tags, y_pred, "Repetition"))
print("Appeal_to_fear-prejudice: ", recall(test2_tags, y_pred, "Appeal_to_fear-prejudice"))
print("Slogans: ", recall(test2_tags, y_pred, "Slogans"))
print("Black-and-White_Fallacy: ", recall(test2_tags, y_pred, "Black-and-White_Fallacy"))
print("Loaded_Language: ", recall(test2_tags, y_pred, "Loaded_Language"))
print("Flag-Waving: ", recall(test2_tags, y_pred, "Flag-Waving"))
print("Name_Calling,Labeling: ", recall(test2_tags, y_pred, "Name_Calling,Labeling"))
print("Whataboutism,Straw_Men,Red_Herring: ", recall(test2_tags, y_pred, "Whataboutism,Straw_Men,Red_Herring"))
print("Causal_Oversimplification: ", recall(test2_tags, y_pred, "Causal_Oversimplification"))
print("Exaggeration,Minimisation: ", recall(test2_tags, y_pred, "Exaggeration,Minimisation"))
print("Bandwagon,Reductio_ad_hitlerum: ", recall(test2_tags, y_pred, "Bandwagon,Reductio_ad_hitlerum"))
print("Thought-terminating_Cliches: ", recall(test2_tags, y_pred, "Thought-terminating_Cliches"))

## C. Comparative Results:

Part 2 Results:

Logistic Regression: Accuracy : 0.56 Precision (Best Class - Name_Calling,Labeling): 0.70 Recall (Best Class - Loaded_Language): 0.88

Decision Tree: Accuracy: 0.47 Precision (Best Class - Name_Calling,Labeling): 0.63 Recall (Best Class - Loaded_Language): 0.72

Logistic Regression: + NLP: Accuracy : 0.51 Precision (Best Class - Appeal_to_fear-prejudice): 0.58 Recall (Best Class - Loaded_Language): 0.88

Decision Tree + NLP: Accuracy: 0.44 Precision (Best Class - Loaded_Language): 0.59 Recall (Best Class - Flag-Waving): 0.72

## 8. Conclusion 

As expected, Logistic Regression performed better than Decision Trees on Task 1, However on Task 2 (Before feature engineering) they had a similar performance.

After Feature engineering, Logistic Regression and Decision Trees still had similar performance. Therefore, overall, Logistic Regression performed better than Decision Trees because in Task 1, it performed significantly better than Decision Trees.

## 9. References

[0] : https://propaganda.qcri.org/semeval2020-task11/

[1] : https://blog.bigml.com/2016/09/28/logistic-regression-versus-decision-trees/

[2] : https://stackoverflow.com/questions/678236/how-to-get-the-filename-without-the-extension-from-a-path-in-python

[3] : https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

[4] : https://datascience.stackexchange.com/questions/17516/how-to-deal-with-string-labels-in-multi-class-classification-with-keras